# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 8:30AM,248005,10,8614967-SD,Eywa Pharma Inc.,Released
1,Oct 6 2022 8:06AM,248014,19,ADV80006285,"AdvaGen Pharma, Ltd",Released
2,Oct 6 2022 8:06AM,248014,19,ADV80006286,"AdvaGen Pharma, Ltd",Released
3,Oct 6 2022 8:06AM,248014,19,ADV80006289,"AdvaGen Pharma, Ltd",Released
4,Oct 6 2022 8:06AM,248014,19,ADV80006292,"AdvaGen Pharma, Ltd",Released
5,Oct 6 2022 8:06AM,248014,19,ADV80006293,"AdvaGen Pharma, Ltd",Released
6,Oct 6 2022 8:06AM,248014,19,ADV80006294,"AdvaGen Pharma, Ltd",Released
7,Oct 6 2022 8:01AM,248013,10,Enova-10773,Emerginnova,Released
8,Oct 6 2022 8:01AM,248013,10,DNMD-24424,Emerginnova,Released
9,Oct 6 2022 8:01AM,248013,10,Enova-10774,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,247996,Released,5
21,247999,Released,4
22,248005,Released,1
23,248013,Released,4
24,248014,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247996,NaN,NaN,5.0
247999,NaN,NaN,4.0
248005,NaN,NaN,1.0
248013,NaN,NaN,4.0
248014,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247900,23.0,10.0,39.0
247916,0.0,0.0,19.0
247918,0.0,0.0,1.0
247926,13.0,0.0,2.0
247964,23.0,8.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247900,23,10,39
247916,0,0,19
247918,0,0,1
247926,13,0,2
247964,23,8,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247900,23,10,39
1,247916,0,0,19
2,247918,0,0,1
3,247926,13,0,2
4,247964,23,8,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247900,23,10,39
1,247916,,,19
2,247918,,,1
3,247926,13,,2
4,247964,23,8,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.
1,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd"
7,Oct 6 2022 8:01AM,248013,10,Emerginnova
11,Oct 5 2022 4:04PM,247999,19,"Innogenix, LLC"
15,Oct 5 2022 3:56PM,247996,19,"Innogenix, LLC"
20,Oct 5 2022 3:46PM,247995,19,"Innogenix, LLC"
25,Oct 5 2022 3:37PM,247994,19,"Innogenix, LLC"
30,Oct 5 2022 3:19PM,247993,10,ISDIN Corporation
48,Oct 5 2022 3:16PM,247991,10,"Snap Medical Industries, LLC"
49,Oct 5 2022 2:31PM,247985,10,"Virtue Rx, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,,,1
1,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",,,6
2,Oct 6 2022 8:01AM,248013,10,Emerginnova,,,4
3,Oct 5 2022 4:04PM,247999,19,"Innogenix, LLC",,,4
4,Oct 5 2022 3:56PM,247996,19,"Innogenix, LLC",,,5
5,Oct 5 2022 3:46PM,247995,19,"Innogenix, LLC",,,5
6,Oct 5 2022 3:37PM,247994,19,"Innogenix, LLC",,,5
7,Oct 5 2022 3:19PM,247993,10,ISDIN Corporation,,,18
8,Oct 5 2022 3:16PM,247991,10,"Snap Medical Industries, LLC",,,1
9,Oct 5 2022 2:31PM,247985,10,"Virtue Rx, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1,,
1,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6,,
2,Oct 6 2022 8:01AM,248013,10,Emerginnova,4,,
3,Oct 5 2022 4:04PM,247999,19,"Innogenix, LLC",4,,
4,Oct 5 2022 3:56PM,247996,19,"Innogenix, LLC",5,,
5,Oct 5 2022 3:46PM,247995,19,"Innogenix, LLC",5,,
6,Oct 5 2022 3:37PM,247994,19,"Innogenix, LLC",5,,
7,Oct 5 2022 3:19PM,247993,10,ISDIN Corporation,18,,
8,Oct 5 2022 3:16PM,247991,10,"Snap Medical Industries, LLC",1,,
9,Oct 5 2022 2:31PM,247985,10,"Virtue Rx, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1,,
1,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6,,
2,Oct 6 2022 8:01AM,248013,10,Emerginnova,4,,
3,Oct 5 2022 4:04PM,247999,19,"Innogenix, LLC",4,,
4,Oct 5 2022 3:56PM,247996,19,"Innogenix, LLC",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
2,Oct 6 2022 8:01AM,248013,10,Emerginnova,4.0,NaN,NaN
3,Oct 5 2022 4:04PM,247999,19,"Innogenix, LLC",4.0,NaN,NaN
4,Oct 5 2022 3:56PM,247996,19,"Innogenix, LLC",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
2,Oct 6 2022 8:01AM,248013,10,Emerginnova,4.0,0.0,0.0
3,Oct 5 2022 4:04PM,247999,19,"Innogenix, LLC",4.0,0.0,0.0
4,Oct 5 2022 3:56PM,247996,19,"Innogenix, LLC",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2231693,96.0,10.0,27.0
15,247926,2.0,0.0,13.0
19,1735945,43.0,8.0,23.0
22,495937,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2231693,96.0,10.0,27.0
1,15,247926,2.0,0.0,13.0
2,19,1735945,43.0,8.0,23.0
3,22,495937,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,96.0,10.0,27.0
1,15,2.0,0.0,13.0
2,19,43.0,8.0,23.0
3,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,15,Released,2.0
2,19,Released,43.0
3,22,Released,0.0
4,10,Executing,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,22
Status,,,,
Completed,27.0,13.0,23.0,0.0
Executing,10.0,0.0,8.0,2.0
Released,96.0,2.0,43.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,22
0,Completed,27.0,13.0,23.0,0.0
1,Executing,10.0,0.0,8.0,2.0
2,Released,96.0,2.0,43.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,22
0,Completed,27.0,13.0,23.0,0.0
1,Executing,10.0,0.0,8.0,2.0
2,Released,96.0,2.0,43.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()